In [1]:
#pip install findspark
#/usr/local/opt/apache-spark/libexec/sbin
#./start-thriftserver.sh

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark").config('spark.driver.memory', '32g').config("hive.server2.thrift.port", 10000).config("spark.sql.hive.thriftServer.singleSession", True).enableHiveSupport().getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/02 20:01:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType,TimestampType
from pyspark.sql.types import DateType
from pyspark.sql.functions import *

schema = StructType([
    StructField("ID", IntegerType()),
    StructField("Speed", DoubleType()),
    StructField("TravelTime", IntegerType()),
    StructField("Status", IntegerType()),
    StructField("timedate", StringType()),
    StructField("LinkId", IntegerType()),
    StructField("LinkPoints", StringType()),
    StructField("EncodedLinkPoints", StringType()),
    StructField("EncodedPolyLineLvls", StringType()),
    StructField("Owner", StringType()),
    StructField("TranscomId", IntegerType()),
    StructField("Borough", StringType()),
    StructField("Link_Name", StringType())
])

schema1 = "ID bigint, Speed float, TravelTime int, Status int, TimeDate string, LinkId int, LinkPoints string, EncodedLinkPoints string, EncodedPolyLineLvls string, Owner string, TranscomId int, Borough string,Link_Name string"
df2 = spark.read.format("csv").option("header", "true").schema(schema).load("file:/Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv")

df2 = df2.withColumn("timedate", to_timestamp("timedate", 'MM/dd/yyyy hh:mm:ss a'))
df2.na.drop()
df2.createOrReplaceTempView("dataset")
df2.count()
df2.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Speed: double (nullable = true)
 |-- TravelTime: integer (nullable = true)
 |-- Status: integer (nullable = true)
 |-- timedate: timestamp (nullable = true)
 |-- LinkId: integer (nullable = true)
 |-- LinkPoints: string (nullable = true)
 |-- EncodedLinkPoints: string (nullable = true)
 |-- EncodedPolyLineLvls: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- TranscomId: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Link_Name: string (nullable = true)



In [ ]:
['LINKID', 'LINK_POINTS', 'ENCODED_POLY_LINE', 'ENCODED_POLY_LINE_LVLS']

In [12]:
columns1= "LINKID, LINKPOINTS, EncodedLinkPoints, EncodedPolyLineLvls"
select1 = "select "+columns1+", count(LinkPoints)"
from1 = " from dataset"
groupby2= " group by "+columns1
orderby = " order by year(timedate) desc"
query1 = select1 + from1 + groupby2 #+ orderby
query1


'select LINKID, LINKPOINTS, EncodedLinkPoints, EncodedPolyLineLvls, count(LinkPoints) from dataset group by LINKID, LINKPOINTS, EncodedLinkPoints, EncodedPolyLineLvls'

In [13]:
linkpoints = spark.sql(query1)
linkpoints.show()

22/04/02 20:05:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS
 Schema: LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls
Expected: LinkId but found: LINK_ID
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


+-------+--------------------+--------------------+--------------------+-----------------+
| LINKID|          LINKPOINTS|   EncodedLinkPoints| EncodedPolyLineLvls|count(LinkPoints)|
+-------+--------------------+--------------------+--------------------+-----------------+
|4620315|40.84667,-73.9318...|uzhxFfyfbMcCbIsGf...|      BBBBBBBBBBBBBB|            31919|
|4616337|40.74047,-74.0092...|}btwFx\\\\\\|ubMs...|       BBBBBBBBBBBBB|            32375|
|4616223|40.6756,-74.841 4...|omgwFfhtbMeDnAaDb...|    BBBBBBBBBBBBBBBB|             5080|
|4616203|40.52581,-74.2303...|iejvF\\\\\\\\\\\\...|     BBBBBBBBBBBBBBB|             6358|
|4616259|40.7279205,-73.83...|otqwFbosaMlEuBbBm...|BBBBBBBBBBBBBBBBB...|             9186|
|4620315|40.84667,-73.9318...|uzhxFfyfbMcCbIsGf...|      BBBBBBBBBBBBBB|             5676|
|4616210|40.63092,-74.1459...|gv~vF~rpcMhF@bE[b...|BBBBBBBBBBBBBBBBB...|            84058|
|4616202|40.56058,-74.1995...|s~pvFjb{cMfGtHxCr...|      BBBBBBBBBBBBBB|             5372|

In [14]:
linkpoints.count()

22/04/02 20:07:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS
 Schema: LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls
Expected: LinkId but found: LINK_ID
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


1868

In [15]:
linkpoints.write.format('csv').mode('overwrite').save('file:/Users/minoseah629/Desktop/ait614/linkpoint1.csv')

22/04/02 20:08:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: LINK_ID, LINK_POINTS, ENCODED_POLY_LINE, ENCODED_POLY_LINE_LVLS
 Schema: LinkId, LinkPoints, EncodedLinkPoints, EncodedPolyLineLvls
Expected: LinkId but found: LINK_ID
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


In [14]:
spark.stop()